In [ ]:
#code to prepare data for unet training 

import numpy as np
import pandas as pd
import os
from obspy import read

# Define your paths
catalog_file = '/home/aswin/Downloads/space_apps_2024_seismic_detection/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
data_path = '/home/aswin/Downloads/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA'
output_path = '/home/aswin/Downloads/space_apps_2024_seismic_detection/output'

# Load catalog
catalog = pd.read_csv(catalog_file)

# Loop through catalog rows and process each shot
for index, row in catalog.iterrows():
    filename = row['filename']
    event_time = row['time_abs(%Y-%m-%dT%H:%M:%S.%f)']
    
    # Construct the full path to the CSV and MSEED files
    csv_file = os.path.join(data_path, filename + '.csv')
    mseed_file = os.path.join(data_path, filename + '.mseed')
    
    # Load CSV file (velocity data)
    if os.path.exists(csv_file):
        csv_data = pd.read_csv(csv_file)
        
        # Extract velocity data and convert to numpy array
        velocities = csv_data['velocity(m/s)'].values
        # You can reshape or process it further if necessary
        npy_data = velocities.reshape(-1, 1)  # Adjust reshape based on your needs
        
        # Save the velocity data as a .npy file
        npy_file = os.path.join(output_path, filename + '.npy')
        np.save(npy_file, npy_data)
        
        # Extract the first break time from catalog and save as a .txt file
        first_break_time = row['time_rel(sec)']  # First break picking time (relative time)
        txt_file = os.path.join(output_path, filename + '.txt')
        with open(txt_file, 'w') as f:
            f.write(str(first_break_time))
        
        # Load MSEED file if needed for additional processing or validation
        if os.path.exists(mseed_file):
            st = read(mseed_file)
            print(f"Loaded MSEED for {filename}")
        else:
            print(f"MSEED file missing for {filename}")
    
    else:
        print(f"CSV file missing for {filename}")

    print(f"Processed {filename}: .npy and .txt saved.")

